# My Portfolio

## Import list of stocks

In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import xlwings as xw

# Load the Excel file from 'Sheet1' into a DataFrame
df = pd.read_excel('datasets/AILabsFinance_Offerings.xlsx', sheet_name='Symbols')

benchmark = df['Benchmark'].iloc[0]
print(f'Benchmark selected: {benchmark}')
df = df[['Symbol','Asset']]
df

Benchmark selected: ^NSEI


,Symbol,Asset
0,HDFCBANK.NS,Stock
1,TCS.NS,Stock
2,ASIANPAINT.NS,Stock
3,RELIANCE.NS,Stock
4,INFY.NS,Stock
5,HINDUNILVR.NS,Stock


## Asset allocation - Donut Plot

In [7]:
# Define asset categories and initialize with zero
all_assets = ['Stock', 'ETF', 'Cash']
asset_dict = {asset: 0 for asset in all_assets}

# Update with actual data from DataFrame
asset_counts = df['Asset'].value_counts(normalize=True) * 100
asset_dict.update(asset_counts)

# Prepare labels, sizes, and other chart properties
labels, sizes = list(asset_dict.keys()), list(asset_dict.values())
colors = ['#FFA500', '#FFD700', '#FF6347', '#FF1493', '#FF4500']
explode = [0.05 if label == 'Stock' else 0 for label in labels]

# Create donut chart
fig, ax = plt.subplots(figsize=(5, 5))

# Reduce white space with tighter layout
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
ax.set_position([0, 0, 1, 1])  # Fill the entire space of the plot

# Plot the pie chart
wedges, _ = ax.pie(sizes, colors=colors, startangle=90, explode=explode, 
                   wedgeprops={'width': 0.3, 'edgecolor': 'white'})

# Donut shape
centre_circle = plt.Circle((0, 0), 0.70, fc='white')
ax.add_artist(centre_circle)

# Equal aspect ratio ensures the pie chart is a circle
plt.axis('equal')

# Title and legend
# plt.title('My Asset Allocation', fontsize=16)
legend_labels = [f'{label}: {size:.2f}%' for label, size in zip(labels, sizes)]
ax.legend(wedges, legend_labels, loc="center left",fontsize=26, bbox_to_anchor=(1, 0, 0.5, 1))

# Save the figure to Excel using xlwings
wb = xw.Book('datasets/AILabsFinance_Offerings.xlsx')
sht = wb.sheets['Symbols']  # Explicitly reference Sheet1

# Remove any existing picture with the same name
for pic in sht.pictures:
    if pic.name == 'Asset Allocation':
        pic.api.Delete()

# Add new picture
sht.pictures.add(fig, name='Asset Allocation', update=True, 
                 left=sht.range('B20').left, top=sht.range('B2').top, width=300)

plt.close()  # Close the plot after saving


In [11]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Define the tickers and benchmark
tickers = df['Symbol'].to_list()
benchmark = '^NSEI'  # Example benchmark index

# Calculate the start and end dates for the last year
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# Download historical data
tickers_data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
benchmark_data = yf.download(benchmark, start=start_date, end=end_date)['Adj Close']

# Parameters
window = 14  # Rolling window size

# Initialize lists to store metrics
rs_tickers = []
rsr_tickers = []
rsr_roc_tickers = []
rsm_tickers = []

# Compute metrics for each ticker
for ticker in tickers:
    # Calculate Relative Strength (RS)
    rs = 100 * (tickers_data[ticker] / benchmark_data)
    rs_tickers.append(rs)

    # Calculate Relative Strength Ratio (RSR)
    rs_rolling_mean = rs.rolling(window=window).mean()
    rs_rolling_std = rs.rolling(window=window).std(ddof=0)
    rsr = (100 + (rs - rs_rolling_mean) / rs_rolling_std).dropna()
    rsr_tickers.append(rsr)

    # Calculate Rate of Change (ROC) of RSR
    rsr_roc = 100 * ((rsr / rsr.shift(1)) - 1)
    rsr_roc_tickers.append(rsr_roc.dropna())

    # Calculate Relative Strength Momentum (RSM)
    rsm_rolling_mean = rsr_roc.rolling(window=window).mean()
    rsm_rolling_std = rsr_roc.rolling(window=window).std(ddof=0)
    rsm = (101 + ((rsr_roc - rsm_rolling_mean) / rsm_rolling_std)).dropna()
    rsm_tickers.append(rsm)

    # Ensure RSR and RSM have the same indices
    rsr_tickers[-1] = rsr_tickers[-1][rsr_tickers[-1].index.isin(rsm_tickers[-1].index)]
    rsm_tickers[-1] = rsm_tickers[-1][rsm_tickers[-1].index.isin(rsr_tickers[-1].index)]

import pandas as pd

# Create a list to store final scores
final_scores_list = []

for i, ticker in enumerate(tickers):
    # Compute the final score for each ticker
    final_score = rsm_tickers[i].mean() if not rsm_tickers[i].empty else None
    final_scores_list.append({'Symbol': ticker, 'Score': final_score})

# Create a DataFrame from the list
final_scores = pd.DataFrame(final_scores_list)

# Compute the Portfolio RRG Score
portfolio_score = final_scores['Score'].mean()

print("Individual Asset Scores:")
print(final_scores)
print("\nPortfolio RRG Score:", portfolio_score)



[*********************100%%**********************]  6 of 6 completed
[*********************100%%**********************]  1 of 1 completed

Individual Asset Scores:
          Symbol       Score
0    HDFCBANK.NS  101.025162
1         TCS.NS  101.030983
2  ASIANPAINT.NS  101.019881
3    RELIANCE.NS  101.013511
4        INFY.NS  100.979738
5  HINDUNILVR.NS  101.023216

Portfolio RRG Score: 101.01541507235216


In [12]:
tickers

['HDFCBANK.NS',
 'TCS.NS',
 'ASIANPAINT.NS',
 'RELIANCE.NS',
 'INFY.NS',
 'HINDUNILVR.NS']

In [13]:
# Prepare DataFrame for scoring
score_df = pd.DataFrame({
    'Symbol': tickers,
    'RS': [rs_tickers[i].iloc[-1] if not rs_tickers[i].empty else None for i in range(len(tickers))],
    'MOM': [rsm_tickers[i].iloc[-1] if not rsm_tickers[i].empty else None for i in range(len(tickers))]
})

In [14]:
score_df

,Symbol,RS,MOM
0,HDFCBANK.NS,7.076444,102.518904
1,TCS.NS,16.604838,100.061066
2,ASIANPAINT.NS,12.293119,103.211352
3,RELIANCE.NS,11.027971,102.324891
4,INFY.NS,7.652344,100.098128
5,HINDUNILVR.NS,10.920931,101.076780


## Compare portfolio

In [11]:
import os
import platform
import sys
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta
import time
import pandas as pd


sys.path.append('../../FinRecipes/')                                       # comment this line for server

from preprocess import Load_n_Preprocess
from models import buy_hold_portfolio_return
from plots import plot_returns_drawdown

df = pd.read_excel('datasets/AILabsFinance_Offerings.xlsx',sheet_name='Symbols')

TICKERS_LIST = df['Symbol'].to_list()
start_time = time.time()
start_download = time.time()
PERIOD = 365*1
END_DATE = dt.datetime.now()
START_DATE = END_DATE - dt.timedelta(days= PERIOD)

LP = Load_n_Preprocess(TICKERS_LIST, START_DATE, END_DATE)
df_tics_daily = LP.download_yfinance(is_live = True)
df_tics_daily = LP.clean_daily_data(df_tics_daily, missing_values_allowed = 0.01)
TICKERS_LIST = df_tics_daily['tic'].unique().tolist()
print(len(TICKERS_LIST))
# df_tics_daily.to_hdf(PATH_DAILY_DATA,"df",mode = 'w')
df_return_bnh = buy_hold_portfolio_return(df_tics_daily)

# ============================= Prepare data for to plot Buy-Hold returns  ======================
df_return_bnh = df_return_bnh.reset_index()
df_return_bnh.columns = ['date','Buy_Hold_Returns']
df_return_bnh['date'] = pd.to_datetime(df_return_bnh['date'])
df_return_bnh = df_return_bnh.set_index(['date'])

# # ===============================     Plotting  ====================================
INDICES_LIST = ['^NSEI']
LP = Load_n_Preprocess(INDICES_LIST, START_DATE, END_DATE)
df_indices_daily = LP.download_yfinance(is_live = True)
df_indices_close = pd.pivot_table(df_indices_daily, values ='close', index = 'date',columns='tic')
df_indices_return = df_indices_close.pct_change().fillna(0)
# df_return_bnh['Portfolio_Trend'] = df_return_bnh['Buy_Hold_returns'].rolling(window=TREND_WINDOW).mean()
df_return_bnh = df_return_bnh.merge(df_indices_return,how='left',on = 'date')
# df_return_bnh = df_return_bnh.drop(columns=['Portfolio_Trend'])

df_return_bnh = df_return_bnh.rename(columns = {'Buy_Hold_Returns':'Portfolio 1','^NSEI': 'NIFTY 50'})
df_return_bnh = df_return_bnh[['Portfolio 1','NIFTY 50']]

plot_returns_drawdown(df_return_bnh, tickers_list = [], filename = 'datasets/Portfolio_BnH', period = 'daily', name_stock_world = None)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Shape of DataFrame:  (1464, 8)
Number of Tickers: 6
[] are removed due to missing data.
Duplicate test: there is no duplicate rows.
6
Shape of DataFrame:  (244, 8)
Number of Tickers: 1


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data (replace these with actual portfolio and Nifty data)
sectors = ['Communication Services', 'Consumer Discretionary', 'Consumer Staples', 'Energy', 
           'Financials', 'Health Care', 'Industrials', 'Utilities', 'Information Technology', 'Materials']

nifty_weights = [2.59, 6.74, 10.04, 14.24, 28.19, 3.6, 3.0, 1.83, 20.82, 8.97]  # Nifty sector weights
portfolio_weights = [2.5, 5.5, 9.0, 12.5, 35, 19.5, 12.3, 0, 3, 0]  # Portfolio sector weights

# Create the plot
fig, ax = plt.subplots(figsize=(8, 6))

# Positioning data
index = np.arange(len(sectors))
bar_width = 0.4

# Plot Nifty and Portfolio weights
ax.barh(index - bar_width/2, nifty_weights, bar_width, label='Nifty', color='blue', alpha=0.6)
ax.barh(index + bar_width/2, portfolio_weights, bar_width, label='Portfolio', color='red', alpha=0.6)

# Labels and titles
ax.set_xlabel('Percentage (%)')
ax.set_title('Sector Allocation: Nifty vs Portfolio')
ax.set_yticks(index)
ax.set_yticklabels(sectors)

# Adding legend
ax.legend()

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
df_tics_daily